# Assignment 1: Preprocessing and Text Classification

Student Name: Jiexin Liu

Student ID: 1044804

# General Info

<b>Due date</b>: Sunday, 28 March 2021 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 9% of mark for class (with 8% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/121115/pages/using-jupyter-notebook-and-python?module_item_id=2681264) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be working with a collection tweets. The task is to predict the geolocation (country) where the tweet comes from. This homework involves writing code to preprocess data and perform text classification.

# Preprocessing (4 marks)

**Instructions**: Download the data (as1-data.json) from Canvas and put it in the same directory as this iPython notebook. Run the code below to load the json data. This produces two objects, `x` and `y`, which contains a list of  tweets and corresponding country labels (it uses the standard [2 letter country code](https://www.iban.com/country-codes)) respectively. **No implementation is needed.**

In [1]:
import json

x = []
y = []
data = json.load(open("as1-data.json"))
for k, v in data.items():
    x.append(k)
    y.append(v)
    
print("Number of tweets =", len(x))
print("Number of labels =", len(y))
print("\nSamples of data:")
for i in range(10):
    print("Country =", y[i], "\tTweet =", x[i])

assert(len(x) == 943)
assert(len(y) == 943)

Number of tweets = 943
Number of labels = 943

Samples of data:
Country = us 	Tweet = @Addictd2Success thx u for following
Country = us 	Tweet = Let's just say, if I were to ever switch teams, Khalesi would be top of the list. #girlcrush
Country = ph 	Tweet = Taemin jonghyun!!! Your birits make me go~ http://t.co/le8z3dntlA
Country = id 	Tweet = depart.senior 👻 rapat perdana (with Nyayu, Anita, and 8 others at Ruang Aescullap FK Unsri Madang) — https://t.co/swRALlNkrQ
Country = ph 	Tweet = Done with internship with this pretty little lady!  (@ Metropolitan Medical Center w/ 3 others) [pic]: http://t.co/1qH61R1t5r
Country = gb 	Tweet = Wow just Boruc's clanger! Haha Sunday League stuff that, Giroud couldn't believe his luck! #clown
Country = my 	Tweet = I'm at Sushi Zanmai (Petaling Jaya, Selangor) w/ 5 others http://t.co/bcNobykZ
Country = us 	Tweet = Mega Fest!!!! Its going down🙏🙌  @BishopJakes
Country = gb 	Tweet = @EllexxxPharrell wow love the pic babe xx
Country = us 	Tweet = You 

### Question 1 (1.0 mark)

**Instructions**: Next we need to preprocess the collected tweets to create a bag-of-words representation. The preprocessing steps required here are: (1) tokenize each tweet into individual word tokens (using NLTK `TweetTokenizer`); (2) lowercase all words; (3) remove any word that does not contain any English alphabets (e.g. {_hello_, _#okay_, _abc123_} would be kept, but not {_123_, _!!_}) and (4) remove stopwords (based on NLTK `stopwords`). An empty tweet (after preprocessing) and its country label should be **excluded** from the output (`x_processed` and `y_processed`).

**Task**: Complete the `preprocess_data(data, labels)` function. The function takes **a list of tweets** and **a corresponding list of country labels** as input, and returns **two lists**. For the first list, each element is a bag-of-words representation of a tweet. For the second list, each element is a corresponding country label. Note that while we do not need to preprocess the country labels (`y`), we need to have a new output list (`y_processed`) because some tweets maybe removed after the preprocessing (due to having an empty set of bag-of-words).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [15]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import re

tt = TweetTokenizer(preserve_case=False)  # lowercase all words
stopwords = set(stopwords.words('english')) #note: stopwords are all in lowercase

def preprocess_data(data, labels):
    
    ###
    # Your answer BEGINS HERE
    ###
    
    ## Tokenize each tweet
    token_list = [tt.tokenize(tweet) for tweet in data]
    
    ## Only keep the work tokens containing letters and remove stopwords
    label_removal_index = []
    BOW_list = []
    
    for index, tweet_tokens in enumerate(token_list):
        tokens_dict = {}
        tweet_tokens = [token for token in tweet_tokens 
                               if (re.search('[A-Za-z]', token) 
                                   and token not in stopwords )]
        if len(tweet_tokens):
            for token in tweet_tokens:
                if token not in tokens_dict.keys():
                    tokens_dict.update({token: 1})
                else:
                    tokens_dict.update({token: tokens_dict.get(token) + 1})
            BOW_list.append(tokens_dict)
        else:
            label_removal_index.append(index)
    
    labels_processed = [label for index, label in enumerate(labels) if index not in label_removal_index]
    
    return BOW_list, labels_processed
    # Your answer ENDS HERE
    ###

x_processed, y_processed = preprocess_data(x, y)

print("Number of preprocessed tweets =", len(x_processed))
print("Number of preprocessed labels =", len(y_processed))
print("\nSamples of preprocessed data:")
for i in range(10):
    print("Country =", y_processed[i], "\tTweet =", x_processed[i])

Number of preprocessed tweets = 943
Number of preprocessed labels = 943

Samples of preprocessed data:
Country = us 	Tweet = {'@addictd2success': 1, 'thx': 1, 'u': 1, 'following': 1}
Country = us 	Tweet = {"let's": 1, 'say': 1, 'ever': 1, 'switch': 1, 'teams': 1, 'khalesi': 1, 'would': 1, 'top': 1, 'list': 1, '#girlcrush': 1}
Country = ph 	Tweet = {'taemin': 1, 'jonghyun': 1, 'birits': 1, 'make': 1, 'go': 1, 'http://t.co/le8z3dntlA': 1}
Country = id 	Tweet = {'depart.senior': 1, 'rapat': 1, 'perdana': 1, 'nyayu': 1, 'anita': 1, 'others': 1, 'ruang': 1, 'aescullap': 1, 'fk': 1, 'unsri': 1, 'madang': 1, 'https://t.co/swRALlNkrQ': 1}
Country = ph 	Tweet = {'done': 1, 'internship': 1, 'pretty': 1, 'little': 1, 'lady': 1, 'metropolitan': 1, 'medical': 1, 'center': 1, 'w': 1, 'others': 1, 'pic': 1, 'http://t.co/1qH61R1t5r': 1}
Country = gb 	Tweet = {'wow': 1, "boruc's": 1, 'clanger': 1, 'haha': 1, 'sunday': 1, 'league': 1, 'stuff': 1, 'giroud': 1, 'believe': 1, 'luck': 1, '#clown': 1}
Countr

**For your testing**:

In [3]:
assert(len(x_processed) == len(y_processed))
assert(len(x_processed) > 800)

**Instructions**: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. Run the code below to collect all unique hashtags in the preprocessed data. **No implementation is needed.**



In [4]:
def get_all_hashtags(data):
    hashtags = set([])
    for d in data:
        for word, frequency in d.items():
            if word.startswith("#") and len(word) > 1:
                hashtags.add(word)
    return hashtags

hashtags = get_all_hashtags(x_processed)
print("Number of hashtags =", len(hashtags))
print(sorted(hashtags))

Number of hashtags = 425
['#100percentpay', '#1stsundayofoctober', '#1yearofalmostisneverenough', '#2011prdctn', '#2015eebritishfilmacademyawards', '#2k16', '#2littlebirds', '#365picture', '#5sosacousticatlanta', '#5sosfam', '#8thannualpubcrawl', '#affsuzukicup', '#aflpowertigers', '#ahimacon14', '#aim20', '#airasia', '#allcity', '#alliswell', '#allwedoiscurls', '#amazing', '#anferneehardaway', '#ariona', '#art', '#arte', '#artwork', '#ashes', '#asian', '#asiangirl', '#askcrawford', '#askherforfback', '#askolly', '#asksteven', '#at', '#australia', '#awesome', '#awesomepict', '#barcelona', '#bart', '#bayofislands', '#beautiful', '#bedimages', '#bell', '#beringmy', '#bettybooppose', '#bff', '#big', '#bigbertha', '#bigbreakfast', '#blackhat', '#blessedmorethanicanimagine', '#blessedsunday', '#blogtourambiente', '#bluemountains', '#bonekachika', '#boomtaob', '#booyaa', '#bored', '#boredom', '#bradersisterhood', '#breaktime', '#breedingground', '#bringithomemy', '#brooksengland', '#burgers'

### Question 2 (1.0 mark)

**Instructions**: Our task here to tokenize the hashtags, by implementing the **MaxMatch algorithm** discussed in class.

NLTK has a list of words that you can use for matching, see starter code below (`words`). Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words (`words`) includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatizer before matching (provided by the function `lemmatize(word)`). Note that the list of words (`words`) is the only source that you'll use for matching (i.e. you do not need to find  other external word lists). If you are unable to make any longer match, your code should default to matching a single letter.

For example, given "#newrecord", the algorithm should produce: \["#", "new", "record"\].

**Task**: Complete the `tokenize_hashtags(hashtags)` function by implementing the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [5]:
from nltk.corpus import wordnet

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK
words = set([ word.lower() for word in words ]) #lowercase all the words for better matching

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

# The program will run 5 seconds or so
def tokenize_hashtags(hashtags):
    tokenized_hashtags = {}

    for hashtag in hashtags:
        i = 0
        tokens = []
        hashtag_text = hashtag[1:]
        hashtag_text = lemmatize(hashtag_text)
        matched_words = []
        non_words = ''
        wordCheckFlag = False

        for word in words:
            if word in hashtag_text:
                matched_words.append(word)

        while i < len(hashtag_text):
            maxMatchedWord = ""
            for j in range(i, len(hashtag_text)):
                tempWord = hashtag_text[i:j + 1]
                if tempWord in matched_words and len(tempWord) > len(maxMatchedWord):
                    maxMatchedWord = tempWord
            if len(maxMatchedWord):
                
                if wordCheckFlag:
                    tokens.append(non_words)
                    non_words = ''
                    wordCheckFlag = False

                i = i + len(maxMatchedWord)
                tokens.append(maxMatchedWord)
            else:
                wordCheckFlag = True
                non_words = non_words + hashtag_text[i]
                if i == len(hashtag_text) - 1:
                    tokens.append(non_words)
                i = i + 1


        tokens.insert(0, '#')
        tokenized_hashtags.update({hashtag: tokens})

    return tokenized_hashtags



#tokenise hashtags with MaxMatch
tokenized_hashtags = tokenize_hashtags(hashtags)

#print results
for k, v in sorted(tokenized_hashtags.items())[-30:]:
    print(k, v)

#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'vega', 'n']
#veganfood ['#', 'vega', 'n', 'food']
#vegetables ['#', 'vegetable']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'v', 's', 'c', 'o']
#vscocam ['#', 'v', 's', 'coca', 'm']
#walking ['#', 'walk']
#watch ['#', 'watch']
#weare90s ['#', 'wear', 'e', '90', 's']
#wearesocial ['#', 'wear', 'es', 'o', 'c', 'i', 'al']
#white ['#', 'white']
#wings ['#', 'wing']
#wok ['#', 'wo', 'k']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mate']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2014']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'ama', 's']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'w', 's', '13']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [6]:
assert(len(tokenized_hashtags) == len(hashtags))
assert(tokenized_hashtags["#newrecord"] == ["#", "new", "record"])

### Question 3 (1.0 mark)

**Instructions**: Our next task is to tokenize the hashtags again, but this time using a **reversed version of the MaxMatch algorithm**, where matching begins at the end of the hashtag and progresses backwards (e.g. for <i>#helloworld</i>, we would process it right to left, starting from the last character <i>d</i>). Just like before, you should use the provided word list (`words`) for word matching.

**Task**: Complete the `tokenize_hashtags_rev(hashtags)` function by the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [7]:
def tokenize_hashtags_rev(hashtags):
    tokenized_hashtags = {}

    for hashtag in hashtags:
        tokens = []
        hashtag_text = hashtag[1:]
        hashtag_text = lemmatize(hashtag_text)
        matched_words = []
        non_words = ''
        wordCheckFlag = False
        i = len(hashtag_text) - 1

        for word in words:
            if word in hashtag_text:
                matched_words.append(word)

        while i >= 0:
            maxMatchedWord = ""
            for j in range(i, -1, -1):

                tempWord = hashtag_text[j: i + 1]

                if tempWord in matched_words and len(tempWord) > len(maxMatchedWord):
                    maxMatchedWord = tempWord

            if len(maxMatchedWord):

                if wordCheckFlag:
                    non_words = non_words[::-1]
                    tokens.append(non_words)
                    non_words = ''
                    wordCheckFlag = False

                i = i - len(maxMatchedWord)
                tokens.append(maxMatchedWord)
            else:
                wordCheckFlag = True
                non_words = non_words + hashtag_text[i]
                if i == 0:
                    non_words = non_words[::-1]
                    tokens.append(non_words)
                i = i - 1
                
        tokens = tokens[::-1]
        tokens.insert(0, '#')
        tokenized_hashtags.update({hashtag: tokens})

    return tokenized_hashtags

    
#tokenise hashtags with the reversed version of MaxMatch
tokenized_hashtags_rev = tokenize_hashtags_rev(hashtags)

#print results
for k, v in sorted(tokenized_hashtags_rev.items())[-30:]:
    print(k, v)

#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'v', 'e', 'gan']
#veganfood ['#', 'v', 'e', 'gan', 'food']
#vegetables ['#', 'vegetable']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'v', 's', 'c', 'o']
#vscocam ['#', 'v', 's', 'c', 'o', 'cam']
#walking ['#', 'walk']
#watch ['#', 'watch']
#weare90s ['#', 'we', 'are', '90', 's']
#wearesocial ['#', 'we', 'are', 'social']
#white ['#', 'white']
#wings ['#', 'wing']
#wok ['#', 'w', 'ok']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mate']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2014']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'a', 'mas']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'w', 's', '13']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [8]:
assert(len(tokenized_hashtags_rev) == len(hashtags))
assert(tokenized_hashtags_rev["#newrecord"] == ["#", "new", "record"])

### Question 4 (1.0 mark)

**Instructions**: The two versions of MaxMatch will produce different results for some of the hashtags. For a hastag that has different results, our task here is to use a **unigram language model** (lecture 3) to score them to see which is better. Recall that in a unigram language model we compute P(<i>#</i>, <i>hello</i>, <i>world</i> = P(<i>#</i>)\*P(<i>hellow</i>)\*P(<i>world</i>).

You should: (1) use the NLTK's Brown corpus (`brown_words`) for collecting word frequencies (note: the words are already tokenised so no further tokenisation is needed); (2) lowercase all words in the corpus; (3) use add-one smoothing when computing the unigram probabilities; and (4) work in the log space to prevent numerical underflow.

**Task**: Build a unigram language model with add-one smoothing using the word counts from the Brown corpus. Iterate through the hashtags, and for each hashtag where MaxMatch and reversed MaxMatch produce different results, print the following: (1) the hashtag; (2) the results produced by MaxMatch and reversed MaxMatch; and (3) the log probability of each result as given by the unigram language model. Note: you **do not** need to print the hashtags where MaxMatch and reversed MaxMatch produce the same results.

An example output:
```
1. #abcd
MaxMatch = [#, a, bc, d]; LogProb = -2.3
Reversed MaxMatch = [#, a, b, cd]; LogProb = -3.5

2. #efgh
MaxMatch = [#, ef, g, h]; LogProb = -4.2
Reversed MaxMatch = [#, e, fgh]; LogProb = -3.1

```

Have a look at the output, and see if the sequences with better language model scores (i.e. less negative) are generally more coherent.

In [9]:
from nltk.corpus import brown
from nltk.probability import FreqDist
import math
import numpy as np

#words from brown corpus
brown_words = brown.words()
vocab_size = len(set(brown_words)) # the size of the vocabulary of the brown corpus
words_size = len(brown_words)

fdist = FreqDist(word.lower() for word in brown_words)  # lowercase all words in the corpus and calculate counts
index_num = 0
diff_list = []


def prob_calc(hashtag):
    prob_maxMatch = 1
    prob_maxMatchRev = 1

    tokens_maxMatch = tokenized_hashtags.get(hashtag)
    tokens_maxMatchRev = tokenized_hashtags_rev.get(hashtag)

    for token in tokens_maxMatch:
        prob_maxMatch = prob_maxMatch * (fdist[token] + 1) / (words_size + vocab_size) # add-one smoothing

    for token in tokens_maxMatchRev:
        prob_maxMatchRev = prob_maxMatchRev * (fdist[token] + 1) / (words_size + vocab_size) # add-one smoothing

    logProb_maxMatch = round(math.log(prob_maxMatch), 1) # log transformation and keep one digit
    logProb_maxMatchRev = round(math.log(prob_maxMatchRev), 1)
    prob_maxMatch = 1
    prob_maxMatchRev = 1

    if logProb_maxMatch != logProb_maxMatchRev:
        return [logProb_maxMatch, logProb_maxMatchRev]
    else:
        return


def display(hashtag, probList, index):
    mes = str(index) + '. ' + hashtag + '\n' + \
          'MaxMatch = ' + str(tokenized_hashtags.get(hashtag)) + '; ' + \
          'LogProb = ' + str(probList[0]) + '\n' + \
          'Reversed MaxMatch = ' + str(tokenized_hashtags_rev.get(hashtag)) + '; ' + \
          'LogProb = ' + str(probList[1]) + '\n'

    return mes


for hashtag in hashtags:
    prob_list = prob_calc(hashtag)
    if prob_list:
        diff_list.append(prob_list[0] - prob_list[1]) # LogProb(MaxMatch) minus LogProb(MaxMatchReverse)
        index_num = index_num + 1
        print(display(hashtag, prob_list, index_num))
    else:
        continue



1. #goingout
MaxMatch = ['#', 'going', 'out']; LogProb = -28.4
Reversed MaxMatch = ['#', 'go', 'in', 'gout']; LogProb = -38.5

2. #blogtourambiente
MaxMatch = ['#', 'blo', 'g', 'tour', 'ambient', 'e']; LogProb = -73.7
Reversed MaxMatch = ['#', 'b', 'log', 'tou', 'ram', 'bien', 'te']; LogProb = -89.1

3. #tcschleissheim
MaxMatch = ['#', 't', 'c', 's', 'c', 'h', 'lei', 's', 'she', 'i', 'm']; LogProb = -111.4
Reversed MaxMatch = ['#', 't', 'c', 's', 'c', 'h', 'l', 'e', 'is', 's', 'hei', 'm']; LogProb = -126.1

4. #cumannanya
MaxMatch = ['#', 'cum', 'anna', 'n', 'ya']; LogProb = -62.7
Reversed MaxMatch = ['#', 'c', 'u', 'mannan', 'ya']; LogProb = -61.0

5. #taintedlove
MaxMatch = ['#', 'taint', 'ed', 'love']; LogProb = -47.3
Reversed MaxMatch = ['#', 'tain', 'ted', 'love']; LogProb = -48.5

6. #instaframeplus
MaxMatch = ['#', 'in', 'st', 'a', 'frame', 'plus']; LogProb = -55.4
Reversed MaxMatch = ['#', 'in', 's', 'ta', 'frame', 'plus']; LogProb = -62.4

7. #ahimacon14
MaxMatch = ['#', 'ah',

In [10]:
print(np.mean(diff_list))   # the output is about 1.23, which means two methods have similar scores but MaxMatch performs
                            # a bit better based on average difference scores

1.2266666666666661


# Text Classification (4 marks)

### Question 5 (1.0 mark)

**Instructions**: Here we are interested to do text classification, to predict the country of origin of a given tweet. The task here is to create training, development and test partitions from the preprocessed data (`x_processed`) and convert the bag-of-words representation into feature vectors.

**Task**: Create training, development and test partitions with a 70%/15%/15% ratio. Remember to preserve the ratio of the classes for all your partitions. That is, say we have only 2 classes and 70% of instances are labelled class A and 30% of instances are labelled class B, then the instances in training, development and test partitions should also preserve this 7:3 ratio. You may use sklearn's builtin functions for doing data partitioning.

Next, turn the bag-of-words dictionary of each tweet into a feature vector. You may also use sklearn's builtin functions for doing this.

You should produce 6 objects: `x_train`, `x_dev`, `x_test` which contain the input feature vectors, and `y_train`, `y_dev` and `y_test` which contain the labels.

In [11]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
import numpy as np

x_train, x_dev, x_test = None, None, None
y_train, y_dev, y_test = None, None, None

v = DictVectorizer(sparse=False)
x_vector = v.fit_transform(x_processed)

x_train, x_temp, y_train, y_temp = train_test_split(x_processed, y_processed, test_size=0.3, stratify=y_processed, random_state=10)
x_dev, x_test, y_dev, y_test = train_test_split(x_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=10)

x_train = v.fit_transform(x_train)
x_dev = v.transform(x_dev)
x_test = v.transform(x_test)



### Question 6 (1.0 mark)

**Instructions**: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation hyper-parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do **not** use cross-validation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the **accuracy** with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output.

**Task**: Implement two text classifiers: Naive Bayes and Logistic Regression. Tune the hyper-parameters of these classifiers and print the task performance (accuracy) for different hyper-parameter settings.

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

###
# Your answer BEGINS HERE
###

# Using cross validation on the development dataset to find the optimal parameters
# scores_NB = []
# for alpha in np.arange(0.1, 10, 0.1):
#     clf_NB_dev = MultinomialNB(fit_prior=False, alpha=alpha)
#     scores_NB.append(cross_val_score(clf_NB_dev, x_dev, y_dev).mean())
# index_optimal = np.argmax(scores_NB)
# alpha_optimal = 0.1* (index_optimal + 1)
# print(alpha_optimal)

print('Multinomial Naive Bayes:')
print('Based on the manual references(scikit-learn docs), the parameters can be tuned are \'fit-prior\' and \'alpha\'.')
print('\'fit-prior\' is set to False due to label imbalance, \'alpha\' is used in smoothing and can be tuned via cross-validation.')
print('According to 5-fold cross validation on the developement dataset, the optimal value is 0.1. The test set accuracy is:')
clf_NB = MultinomialNB(fit_prior=False, alpha=0.1)
clf_NB.fit(x_train, y_train)
print(clf_NB.score(x_test, y_test))
print('When alpha is set to 2, the accuracy is')
clf_NB = MultinomialNB(fit_prior=False, alpha=2)
clf_NB.fit(x_train, y_train)
print(clf_NB.score(x_test, y_test))
print('When alpha is set to 10, the accuracy is')
clf_NB = MultinomialNB(fit_prior=False, alpha=10)
clf_NB.fit(x_train, y_train)
print(clf_NB.score(x_test, y_test))

print('\nLogistic Regression:')
print('Based on manual references, \'lbfgs\' solver is supported for multinomial logistic regression and converge faster for some high-dimensional data.')
print('L2 regularization is carried out by default, the parameters should be tuned here is C which specify how strong the regularization is.')
# Using cross validation on the development dataset to find the optimal parameters
# scores_LR = []
# for c in np.arange(0.1, 10, 0.1):
#     clf_LR_dev = LogisticRegression(multi_class='multinomial', penalty='l2', solver='lbfgs', C=c)
#     scores_LR.append(cross_val_score(clf_LR_dev, x_dev, y_dev).mean())
#     print(c)
# index_optimal = np.argmax(scores_LR)
# c_optimal = 0.1* (index_optimal + 1)
# print(c_optimal) # c_optimal = 2.5

print('c is 2.5, which is the optimal value, the test set accuracy is:')
clf_LR = LogisticRegression(multi_class='multinomial', penalty='l2', solver='lbfgs', C=2.5)
clf_LR.fit(x_train, y_train)
print(clf_LR.score(x_test, y_test))
print('c is 0.1,  the test set accuracy is:')
clf_LR = LogisticRegression(multi_class='multinomial', penalty='l2', solver='lbfgs', C=0.1)
clf_LR.fit(x_train, y_train)
print(clf_LR.score(x_test, y_test))
print('c is 10, the test set accuracy is:')
clf_LR = LogisticRegression(multi_class='multinomial', penalty='l2', solver='lbfgs', C=10)
clf_LR.fit(x_train, y_train)
print(clf_LR.score(x_test, y_test))
print('The variation of accuracy in the test set is subtle when choosing different c. The parameters may be reliant on the dataset.')
    
###
# Your answer ENDS HERE
###

Multinomial Naive Bayes:
Based on the manual references(scikit-learn docs), the parameters can be tuned are 'fit-prior' and 'alpha'.
'fit-prior' is set to False due to label imbalance, 'alpha' is used in smoothing and can be tuned via cross-validation.
According to 5-fold cross validation on the developement dataset, the optimal value is 0.1. The test set accuracy is:
0.29577464788732394
When alpha is set to 2, the accuracy is
0.28169014084507044
When alpha is set to 10, the accuracy is
0.2746478873239437

Logistic Regression:
Based on manual references, 'lbfgs' solver is supported for multinomial logistic regression and converge faster for some high-dimensional data.
L2 regularization is carried out by default, the parameters should be tuned here is C which specify how strong the regularization is.
c is 2.5, which is the optimal value, the test set accuracy is:
0.2746478873239437
c is 0.1,  the test set accuracy is:
0.24647887323943662
c is 10, the test set accuracy is:
0.274647887323

### Question 7 (1.0 mark)

**Instructions**: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both **accuracy** and **macro-averaged F-score** for each classifier. Be sure to label your output. You may use sklearn's inbuilt functions.

**Task**: Compute test performance in terms of accuracy and macro-averaged F-score for both Naive Bayes and Logistic Regression, using their optimal hyper-parameter settings based on their development performance.

In [13]:
###
# Your answer BEGINS HERE
###

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

clf_NB = MultinomialNB(fit_prior=False, alpha=0.1)
clf_LR = LogisticRegression(multi_class='multinomial', penalty='l2', solver='lbfgs', C=2.5)

clf_NB.fit(x_train, y_train)
clf_LR.fit(x_train, y_train)

y_pre_NB = clf_NB.predict(x_test)
y_pre_LR = clf_LR.predict(x_test)


print('Naive Bayes:')
print('The accuracy is:', accuracy_score(y_test, y_pre_NB))
print('The macro-averaged F-score is:', f1_score(y_test, y_pre_NB, average='macro'))

print('\nLogistic Regression:')
print('The accuracy is:', accuracy_score(y_test, y_pre_LR))
print('The macro-averaged F-score is:', f1_score(y_test, y_pre_LR, average='macro'))


###
# Your answer ENDS HERE
###

Naive Bayes:
The accuracy is: 0.29577464788732394
The macro-averaged F-score is: 0.3014040724641213

Logistic Regression:
The accuracy is: 0.2746478873239437
The macro-averaged F-score is: 0.243900165976391


### Question 8 (1.0 mark)

**Instructions**: Print the most important features and their weights for each class for the two classifiers.


**Task**: For each of the classifiers (Logistic Regression and Naive Bayes) you've built in the previous question, print out the top-20 features (words) with the highest weight for each class (countries).

An example output:
```
Classifier = Logistic Regression

Country = au
aaa (0.999) bbb (0.888) ccc (0.777) ...

Country = ca
aaa (0.999) bbb (0.888) ccc (0.777) ...

Classifier = Naive Bayes

Country = au
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...

Country = ca
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...
```

Have a look at the output, and see if you notice any trend/pattern in the words for each country.

In [14]:
###
# Your answer BEGINS HERE
###


metrics_NB = clf_NB.feature_log_prob_
metrics_LR = clf_LR.coef_



features_NB = []
features_LR = []

for label_index in range(0, len(clf_NB.classes_)):
    index_NB = np.argsort(-metrics_NB[label_index])[0:20]
    temp = []
    for index in index_NB:
        temp.append([v.feature_names_[index], metrics_NB[label_index][index]])
    features_NB.append(temp)
    
for label_index in range(0, len(clf_LR.classes_)):
    index_LR = np.argsort(-metrics_LR[label_index])[0:20]
    temp = []
    for index in index_LR:
        temp.append([v.feature_names_[index], metrics_LR[label_index][index]])
    features_LR.append(temp)
    
    
print('Classifier = Logistic Regression')
for label_index in range(0, len(clf_LR.classes_)):
    print('\nCountry = ', clf_LR.classes_[label_index])
    for item in features_LR[label_index]:
        print(item[0], '(', item[1], ') ', end='')
    print()

print('\nClassifier = Naive Bayes')
for label_index in range(0, len(clf_NB.classes_)):
    print('\nCountry = ', clf_NB.classes_[label_index])
    for item in features_NB[label_index]:
        print(item[0], '(', item[1], ') ', end='')
    print()     
        

###
# Your answer ENDS HERE
###

Classifier = Logistic Regression

Country =  au
australia ( 1.5803831509226829 ) melbourne ( 1.3076985324416734 ) #melbourne ( 1.2564130077957747 ) @micksunnyg ( 1.2386881932528513 ) literally ( 1.2297588280451757 ) little ( 1.1584828981205144 ) brilliant ( 1.1192685194372936 ) great ( 0.9800358456939551 ) spammed ( 0.966717326957859 ) instagrams ( 0.966717326957859 ) australian ( 0.9498058992908894 ) done ( 0.9271458267819103 ) #sunrise ( 0.8645226450990786 ) forward ( 0.8413353444816353 ) victoria ( 0.8403073366136744 ) http://t.co/blhj9cMxiT ( 0.8390359699571185 ) @christorrano ( 0.8390359699571185 ) away ( 0.8365258364357832 ) geelong ( 0.8309208363720756 ) freo ( 0.8309208363720756 ) 

Country =  ca
bed ( 1.2432162709035066 ) thing ( 1.2373321695079647 ) hate ( 1.2019500474755114 ) works ( 1.1043451603423735 ) follow ( 1.0173322636358906 ) let's ( 0.9841314374939042 ) next ( 0.9551372617733771 ) @aliclarke ( 0.9395089814829012 ) @cheetahbiatch ( 0.9163704184926477 ) xoxo ( 0.91637